In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
%cd /content

# 切到你的專案資料夾
%cd /content/drive/MyDrive/LSTM_PROGRAM

Mounted at /content/drive
/content
/content/drive/MyDrive/LSTM_PROGRAM


In [3]:
import os
import pandas as pd
import re
from functools import reduce
from collections import defaultdict

# 2. 【用戶自定義區間】
start_date = '2022/05/01'
end_date = '2025/07/01'
start = pd.to_datetime(start_date)
end = pd.to_datetime(end_date)


# 路徑和檔名
folder = '.'
csv_files = [
    './filtered_output/row_group/bonds_day_aligned.csv',
    './filtered_output/row_group/bonds_hour_aligned.csv',
    './filtered_output/row_group/crypto_day_aligned.csv',
    './filtered_output/row_group/crypto_hour_aligned.csv',
    './filtered_output/row_group/stock_day_aligned.csv',
    './filtered_output/row_group/stock_hour_aligned.csv',
    './filtered_output/row_group/others_day_aligned.csv',
    './filtered_output/row_group/others_hour_aligned.csv'
]

def find_date_col(df):
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            return col
    return df.columns[0]

def get_prefix(file):
    if 'bonds_day' in file: return 'bonds_day'
    if 'bonds_hour' in file: return 'bonds_hour'
    if 'crypto_day' in file: return 'crypto_day'
    if 'crypto_hour' in file: return 'crypto_hour'
    if 'stock_day' in file: return 'stock_day'
    if 'stock_hour' in file: return 'stock_hour'
    if 'others_day' in file: return 'others_day'
    if 'others_hour' in file: return 'others_hour'

    return os.path.splitext(os.path.basename(file))[0]

def read_and_clean(file):
    df = pd.read_csv(file)
    date_col = find_date_col(df)
    tried = False
    for fmt in ['%Y-%m-%d %H:%M:%S', '%Y/%m/%d %H:%M', '%Y-%m-%d %H:%M', '%Y/%m/%d %H:%M:%S']:
        try:
            df[date_col] = pd.to_datetime(df[date_col], format=fmt, errors='raise')
            tried = True
            break
        except Exception:
            continue
    if not tried:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    # 強制 floor 到小時，避免交集失敗
    df[date_col] = df[date_col].dt.floor('h')
    df[date_col] = df[date_col].dt.tz_localize(None)
    df = df.rename(columns={date_col: 'DATE'})
    return df


dfs = []
for file in csv_files:
    df = read_and_clean(file)
    date_col = find_date_col(df)
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df = df.dropna(subset=[date_col])
    prefix = get_prefix(file)
    # 將所有非日期欄位加前綴
    rename = {col: f"{col.upper()}" for col in df.columns if col != date_col}
    df = df.rename(columns=rename)
    df = df.rename(columns={date_col: 'DATE'})
    dfs.append(df)

#{prefix}_

# df_shapes = pd.DataFrame([
#     {
#         'idx': i,
#         'rows': df.shape[0],
#         'cols': df.shape[1]
#     }
#     for i, df in enumerate(dfs)
# ])
# print(df_shapes)

# 1) 先把 dfs 和 csv_files 对应起来，生成一个 prefix→df 的 dict
dfs_dict = {}
for file, df in zip(csv_files, dfs):
    prefix = get_prefix(file)       # e.g. 'bonds_day', 'bonds_hour'
    dfs_dict[prefix] = df
    print(dfs_dict[prefix].columns)

# 2) 重组成 asset → {'day': df, 'hour': df}，方便后续两两配对
by_asset = defaultdict(dict)

for prefix, df in dfs_dict.items():
    asset, freq = prefix.rsplit('_', 1)  # 把 'bonds_day' 分成 ['bonds','day']
    by_asset[asset][freq] = df
    #print(by_asset[asset][freq].columns)
    # print(by_asset[asset][freq]['DATE'].dtype)


out_dir = './output1/row_group'
os.makedirs(out_dir, exist_ok=True)

# 先在外面建一个空字典，用来存放 clean 后的小时线
cleaned_hours = {}

for asset, pair in by_asset.items():

    if 'day' in pair and 'hour' in pair:
        print(f"{asset}: day 有 {pair['day'].shape}，hour 有 {pair['hour'].shape}")
    else:
        print(f"⚠️ {asset} 缺少 day 或 hour，检查 get_prefix 是否正确")

    df_day  = pair['day']
    df_hour = pair['hour']

    # 1) 按 DATE 生成字符串，方便比对
    df_day  = df_day.copy()
    df_hour = df_hour.copy()

    df_day['DATE_STR']  = pd.to_datetime(df_day['DATE']).dt.strftime('%Y-%m-%d')
    df_hour['DATE_full'] = pd.to_datetime(df_hour['DATE'])
    df_hour['DATE_STR'] = df_hour['DATE_full'].dt.strftime('%Y-%m-%d')

    # 2) 找到那些 hour 数据里，日期不在 day 里的，单独输出 exclude
    df_hour_exclude = df_hour[~df_hour['DATE_STR'].isin(df_day['DATE_STR'])].copy()
    df_hour_exclude = df_hour_exclude.drop(columns=['DATE_STR'])
    df_hour_exclude = df_hour_exclude.rename(columns={'DATE_full': 'DATE'})
    path_ex = os.path.join(out_dir, f"{asset}_hourly_exclude.csv")
    df_hour_exclude.to_csv(path_ex, index=False, encoding='utf-8-sig')
    print(f"❌ {asset} – 排除掉不在日线中的小时点，共 {len(df_hour_exclude)} 行 → {path_ex}")

    # 3) 保留 clean 部分
    df_hour_clean = df_hour[df_hour['DATE_STR'].isin(df_day['DATE_STR'])].copy()
    df_hour_clean = df_hour_clean.drop(columns=['DATE_STR'])
    df_hour_clean = df_hour_clean.rename(columns={'DATE_full': 'DATE'})
    path_cl = os.path.join(out_dir, f"{asset}_hourly_clean.csv")
    df_hour_clean.to_csv(path_cl, index=False, encoding='utf-8-sig')
    print(f"✅ {asset} – 清洗后小时线对齐日线，共 {len(df_hour_clean)} 行 → {path_cl}")

    # 4) 把 clean 后的 df 存进字典
    cleaned_hours[asset] = df_hour_clean

# # 循环结束后，cleaned_hours 就是一个 { asset: df_hour_clean } 的字典
# # 你可以在此之后再统一对它们做什么，比如：
# for asset, df in cleaned_hours.items():
#     print(f"{asset} 对齐后共有 {df.shape[0]} 行，{df.shape[1]} 列")
#     # 一次性看所有列
#     print(df.dtypes)
#     print('-' * 40)

# print(type(cleaned_hours), type(by_asset))
# print("cleaned_hours 的资产列表：", list(cleaned_hours.keys()))
# print("by_asset 的资产列表：", list(by_asset.keys()))

# 输出目录
out_dir1 = './filtered_output'
os.makedirs(out_dir1, exist_ok=True)

for asset, df_hour in cleaned_hours.items():
    # 1) 删除重复的列（保留第一个出现的）
    df_hour = df_hour.loc[:, ~df_hour.columns.duplicated()]

    mask = (df_hour['DATE'] >= start) & (df_hour['DATE'] <= end)
    df_hour = df_hour.loc[mask]  # 先筛选
    df_hour.loc[:, 'DATE'] = pd.to_datetime(df_hour['DATE'], errors='coerce')

    # 然后导出
    out_path = os.path.join(out_dir1, f"{asset}_hour_clean_period.csv")
    df_hour.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"✅ {asset} 小时数据 ({len(df_hour)} rows) 已保存到 {out_path}")


# 2) 处理日数据
for asset, freq_dict in by_asset.items():
    df_day = freq_dict.get('day')
    if df_day is None:
        print(f"⚠️ {asset} 没有日频数据，跳过")
        continue
    # 确保 DATE 列是 datetime
    df_day['DATE'] = pd.to_datetime(df_day['DATE'])
    # 筛选
    df_filt = df_day[(df_day['DATE'] >= start) & (df_day['DATE'] <= end)]
    # 导出
    out_path = os.path.join(out_dir1, f"{asset}_day_clean_period.csv")
    df_filt.to_csv(out_path, index=False, encoding='utf-8-sig')
    print(f"✅ {asset} 日线数据 ({len(df_filt)} rows) 已保存到 {out_path}")

Index(['DATE', 'SR3_CLOSE', 'ZT1_CLOSE', 'ZF1_CLOSE', 'TN1_CLOSE', 'ZN1_CLOSE',
       'ZB1_CLOSE', 'SR3_VOLUME', 'ZT1_VOLUME', 'ZF1_VOLUME', 'TN1_VOLUME',
       'ZN1_VOLUME', 'ZB1_VOLUME', 'SR3_OPEN', 'ZT1_OPEN', 'ZF1_OPEN',
       'TN1_OPEN', 'ZN1_OPEN', 'ZB1_OPEN', 'SR3_HIGH', 'ZT1_HIGH', 'ZF1_HIGH',
       'TN1_HIGH', 'ZN1_HIGH', 'ZB1_HIGH', 'SR3_LOW', 'ZT1_LOW', 'ZF1_LOW',
       'TN1_LOW', 'ZN1_LOW', 'ZB1_LOW', 'SR3_IS_TRADING', 'ZT1_IS_TRADING',
       'ZF1_IS_TRADING', 'TN1_IS_TRADING', 'ZN1_IS_TRADING', 'ZB1_IS_TRADING'],
      dtype='object')
Index(['DATE', 'SR3_CLOSE', 'ZT1_CLOSE', 'ZF1_CLOSE', 'TN1_CLOSE', 'ZN1_CLOSE',
       'ZB1_CLOSE', 'SR3_VOLUME', 'ZT1_VOLUME', 'ZF1_VOLUME', 'TN1_VOLUME',
       'ZN1_VOLUME', 'ZB1_VOLUME', 'SR3_OPEN', 'ZT1_OPEN', 'ZF1_OPEN',
       'TN1_OPEN', 'ZN1_OPEN', 'ZB1_OPEN', 'SR3_HIGH', 'ZT1_HIGH', 'ZF1_HIGH',
       'TN1_HIGH', 'ZN1_HIGH', 'ZB1_HIGH', 'SR3_LOW', 'ZT1_LOW', 'ZF1_LOW',
       'TN1_LOW', 'ZN1_LOW', 'ZB1_LOW', 'SR3_IS_TRADING', 